In [8]:
#import all required module
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import random
import pandas as pd
import time
import os
import pandas as pd
 
# record start time
start = time.time()

linkedin_home = 'https://www.linkedin.com/'
linkedin_email = "" # place your linkedin login email
linkedin_password =  ""# place your linkedin login password
# company_name = "Adani Green Energy Ltd." # Replace with the name of the company you want to search for
company_name = input("Enter the name of company.")

# output_csv_path = r"C:\Users\30019622\Downloads\Python\Demand Forecasting\Names.csv"

#use when having a list of companies in excel
# company_excel_path = r"C:\Users\30019622\Downloads\Python\Demand Forecasting\Company for Linkedin.xlsx"
# company_df = pd.read_excel(company_excel_path)
# company_list = company_df['Company']
#use below for loop

# for name in company_list:
#     company_name = name

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')

driver = webdriver.Chrome(options=options)
time.sleep(1)
driver.maximize_window()
driver.get(linkedin_home.format()) 
time.sleep(1)

# Linkedin Login
login_email = driver.find_element(By.ID,'session_key')
login_email.send_keys(linkedin_email)
time.sleep(2)
login_pwd = driver.find_element(By.ID,'session_password')
login_pwd.send_keys(linkedin_password)
time.sleep(2)
driver.find_element(By.XPATH,"//button[contains(text(), 'Sign in')]").click()
time.sleep(2)

#open company link in new tab
driver.execute_script("window.open('');")
driver.switch_to.window(driver.window_handles[1])
driver.get(linkedin_home)
time.sleep(10)

#searching company page
search_box = driver.find_element(By.XPATH,"//input[@aria-label='Search']")
time.sleep(5)
search_box.send_keys(company_name)
time.sleep(3)
search_box.send_keys(Keys.DOWN)
time.sleep(3)
search_box.send_keys(Keys.ENTER)
time.sleep(3)
driver.find_element(By.XPATH, "//button[text() = 'Companies']").click()
time.sleep(3)

driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul/li/div/div/div[2]/div[1]/div[1]/div/span/span/a").click()
time.sleep(2)

try:
    link_with_mutual = driver.find_element(By.XPATH,"//a [@class='ember-view org-top-card-secondary-content__see-all-link']").click()
except:
    link_without_mutual = driver.find_element(By.XPATH,"//span[@class='link-without-visited-state t-bold t-black--light']").click()
    
time.sleep(2)

master_url = []
master_exp = []
master_users=[]
exp_data = []
user_data=[]
main_dict = {}

#looping for next button
try:
    for n in range (1):
        driver.switch_to.window(driver.window_handles[1])
        url_list=[]
        counter=1
        time.sleep(2)
        #capturing URL of 10 profiles displayed
        for counter in range(1,11):
            try:
                if driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul/li["+str(counter)+"]/div/div/div[3]/div/button"):
#                     print ("Either Connect or Follow button found. Not private member")
                    profile_url = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul/li["+str(counter)+"]/div/div/div[2]/div[1]/div[1]/div/span[1]/span/a").get_attribute("href")
                    url_list.append(profile_url)

            except:
                try:
                    if driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul/li["+str(counter)+"]/div/div/div[3]/div/div/button"):
#                         print ("Message button found. Not private member")other error
                        profile_url = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul/li["+str(counter)+"]/div/div/div[2]/div[1]/div[1]/div/span[1]/span/a").get_attribute("href")
                        url_list.append(profile_url)
                except:
                    print("Private linkedin profile found. Skipped adding to url list")


        #looping data extraction for 10 URLs captured
        for n in range(len(url_list)):
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[1])
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[2])
            driver.get(url_list[n])
            #print(url_list[n])
            time.sleep(2)
            try:
                user_name = driver.find_element(By.XPATH,"//h1[@class='text-heading-xlarge inline t-24 v-align-middle break-words']").text
                experience_block = driver.find_element(By.XPATH,"//span[text() = 'Experience']//parent::h2//parent::div//parent::div//parent::div//parent::div//parent::section")
                new = experience_block.text.strip().split("\n")
                exp_det1 = [i for n, i in enumerate(new) if i not in new[:n]]
                indices = [i for i, s in enumerate(exp_det1) if 'Present' in s]
                exp_det2 = exp_det1[1:indices[0]+1]
                ans = ''
                for i in exp_det2:
                    ans = ans+ ' '+ i + "\n"
                exp_det = ans.strip()
                #print(exp_det)
                driver.close()
            except:
                print('other error')

            #appending users data to master list
            main_dict["S No."] = n
            main_dict['User Name'] = user_name
            main_dict['URL'] = url_list[n]
            main_dict['Experience'] = exp_det
            master_exp.append(exp_det)
            master_url.append(url_list[n])
            master_users.append(user_name)


        #switching to company page
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, 1080)") 
        time.sleep(2)

        #click on next button
        driver.find_element(By.XPATH,"//button[@aria-label='Next']").click()
        time.sleep(2)

except:
    print("Error occured. People has less than 99 pages")

df = pd.DataFrame()
df['Users']= master_users
df["URL"] = master_url
df["Experience"] = master_exp
df["Comapny_Name"] = company_name
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    #change csv path
# get the current working directory
cwd = os.getcwd()

# create a filename with a timestamp
# filename = 'Excel_File_{}.xlsx'.format(pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M-%S'))
filename = '{}.xlsx'.format(company_name)

# save the file in the current working directory
df.to_excel(os.path.join(cwd, filename), index=False)


print('File saved as:', filename)

Enter the name of company.Adani Green Energy Ltd.
other error
other error
00:02:12.95
File saved as: Adani Green Energy Ltd..xlsx
